In [ ]:
import math

import efd_utils

In [ ]:
location = "summit"

In [ ]:
client = efd_utils.get_client(location)

In [ ]:
cscs = efd_utils.get_cscs(f"{location}_systems.dat")

In [ ]:
summary_state = 4  # STANDBY
time_window = 10.0  # seconds

In [ ]:
for csc in cscs:
    query = efd_utils.get_base_query(columns=["private_sndStamp",
                                              "summaryState"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name="logevent_summaryState")

    # Handle indexed components from base query
    if "WHERE" not in query:
        query += " WHERE"
    query += f" summaryState={summary_state}"
    query += " " + efd_utils.get_time_clause(last=True)

    ss_df = await client.query(query)

    query = efd_utils.get_base_query(columns=["private_sndStamp",
                                              "recommendedSettingsLabels",
                                              "recommendedSettingsVersion"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name="logevent_settingVersions")

    query += efd_utils.get_time_clause(last=True)

    sv_df = await client.query(query)

    print("###########################################################")
    print(f"CSC: {csc.name}")
    try:
        print(f"Time of Summary State: {ss_df.index[0]}")
    except AttributeError:
        print(f"summaryState event not present for {csc.name}")
    try:
        if sv_df.size:
            delta = ss_df.private_sndStamp.values[0] - sv_df.private_sndStamp.values[0]
            if math.fabs(delta) > time_window:
                print(f"Large delay in settingVersions publish: {delta:.1f} seconds")
            rsl = sv_df.recommendedSettingsLabels.values[0]
            rsv = sv_df.recommendedSettingsVersion.values[0]
            print(f"Recommended Settings Labels: {rsl}")
            print(f"Recommended Settings Version: {rsv}")
        else:
            print(f"settingVersions event not present for {csc.name}")
    except AttributeError:
        print(f"settingVersions event not present for {csc.name}")